# Assignment Editor Agent

This notebook illustrates how the very first version of this agent was developed

In [2]:
import sys
import json
from pathlib import Path
from dotenv import load_dotenv

from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import START, END, StateGraph


# Add src to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

from agentic_newsroom.schemas import StoryBrief

# Load environment variables
env_path = project_root / '.env'
load_dotenv(dotenv_path=env_path)

print("✓ Imports successful")
print(f"✓ Loading .env from: {env_path}")

✓ Imports successful
✓ Loading .env from: /Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/.env


## Assignment Editor Prompts

In [3]:
# ============= FROZEN PROMPTS =============

magazine_profile = """ Agentic Newsroom is a magazine of discovery and wonder.
It explores the hidden corners of the planet and the universe, cutting-edge scientific mysteries, strange natural phenomena, ancient civilization, unusual places, and the latest in space exploration.
Our editorial voice blends scientific rigor, vivid storytelling, a touch of wonder, and strong narrative.
Articles are deeply researched, visually descriptive, and rich in detail.
We aim to make complex ideas accessible, without dumbing them down and to evoke awe, curiosity, and sense of scale.
"""

magazine_guardrails = """
Agentic Newsroom adheres to the following editorial standards in all published writing:

1. Agentic Newsroom does not produce or publish any form of hate speech, harassment, racism, sexism, or discriminatory content.
2. Articles may occasionally discuss pseudoscience, fringe theories, myths, or unverified claims when relevant to cultural or historical context. 
   However, Agentic Newsroom does not endorse such claims and always presents them with clear scientific, historical, or factual framing.
3. All information must be presented accurately, responsibly, and with respect for both scientific consensus and ethical journalism.
4. When uncertainty exists, Agentic Newsroom clearly distinguishes between established fact, informed interpretation, and speculation.
"""

assignment_editor_profile = """
Your name is Robert. You are the Assignment Editor for Agentic Newsroom.
You have more than twenty years of experience shaping coverage for major science and geography magazines. 
Your role is to identify stories worth telling and define the editorial direction before reporting begins. 
You think like a strategist: every pitch, idea, and angle must serve Agentic Newsroom's mission of curiosity, scientific literacy, and wonder.

You craft clear, actionable story briefs that specify:
- the scope and angle
- the required reporting questions
- the story type and length
- the intended audience and tone
- the themes or narrative tension

Your job is to give the reporter a direction that is specific, compelling, achievable, and worth publishing.
"""

assignment_editor_prompt = """You are the Assignment Editor for The Agentic Newsroom.
<Magazine Profile>
This is the magazine you work for:
{magazine_profile}
</Magazine Profile>

<Magazine Guardrails>
Strictly adhere to the following editorial standards:
{magazine_guardrails}
</Magazine Guardrails>

<Assignment Editor Profile>
This is your profile:
{assignment_editor_profile}
</Assignment Editor Profile>

<Task>
You are given a an idea of an article and your task is to turn it into a story brief.
It's important that the brief matches the magazine's tone and profile
</Task>

<Instructions>
When you receive a article idea, follow these steps:

1. Think carefully what the audience would want to know about this article and phrase a clear topic based on it
2  Come up with a clear angle that will make the a interesting and engaging for the readers
3. Think about the key questions that the article should answer
4. Estimate the length of the article in words. Use the following as a guide for the length:
 - Short feature or explainer: 400-700 words
 - Medium feature: 700-1000 words
</Instructions>

<Output Format>
When you have come up with the story brief, output it in the following format:
 - Topic
 - Angle
 - Length in words
 - Key questions
</Output Format>
"""

## States and Structured Outputs

In [4]:
# ============= FROZEN SCHEMAS =============

from typing import List, Optional, TypedDict
from pydantic import BaseModel, Field

class StoryBrief(BaseModel):
    """Story brief created by the Assignment Editor."""
    topic: str = Field(..., description="Clear statement of what the story is about")
    angle: str = Field(..., description="The specific approach or perspective to take")
    length: int = Field(..., description="The length of the article in words")
    key_questions: List[str] = Field(..., description="3-5 questions the article should answer")

class NewsroomState(TypedDict):
    """State for the unified newsroom workflow."""
    article_idea: str
    story_brief: Optional[StoryBrief]
    research_package: Optional[object]
    draft_package: Optional[object]
    final_article: Optional[object]
    editor_decision: Optional[object]
    feedback: Optional[str]

## Configuration

In [5]:
model = init_chat_model(model="openai:gpt-5.1", reasoning_effort="medium")
#model = init_chat_model(model="openai:gpt-5-mini", reasoning_effort="minimal")

print("✓ Configuration complete")

✓ Configuration complete


## Nodes

In [6]:
def create_story_brief(state: NewsroomState):
    
    system_msg = SystemMessage(content=assignment_editor_prompt.format(
        magazine_profile=magazine_profile,
        magazine_guardrails=magazine_guardrails,
        assignment_editor_profile=assignment_editor_profile
    ))

    structured_model = model.with_structured_output(StoryBrief)

    response = structured_model.invoke([
        system_msg,
        HumanMessage(content=f"Write a story brief for the following article idea: {state['article_idea']}")
    ])

    return {
        "story_brief": response
    }


    

## Assignment Editor Subgraph

In [7]:
def create_assignment_editor_subgraph():
    graph = StateGraph(NewsroomState)
    
    # Single node: assignment editor
    graph.add_node("create_story_brief", create_story_brief)
    
    # Simple flow: START → assignment_editor → END
    graph.add_edge(START, "create_story_brief")
    graph.add_edge("create_story_brief", END)
    
    return graph.compile()


# instantiate it
assignment_editor_graph = create_assignment_editor_subgraph()

# Optional visualization
print(assignment_editor_graph.get_graph().draw_ascii())

    +-----------+      
    | __start__ |      
    +-----------+      
           *           
           *           
           *           
+--------------------+ 
| create_story_brief | 
+--------------------+ 
           *           
           *           
           *           
      +---------+      
      | __end__ |      
      +---------+      


## Run the Assignment Editor Subgraph

In [8]:
# Raw topic for testing
topic = (
    """Medium length article about the island of Socotra. It should a captivating exploration into this unique island and most distinctive features.
    """
)

# Create initial state with the article idea
initial_state = {
    "article_idea": topic.strip()  # .strip() to remove extra whitespace
}

assignment_editor_graph = create_assignment_editor_subgraph()
result = assignment_editor_graph.invoke(initial_state)  # Fixed: removed list brackets

assignment_editor_graph = create_assignment_editor_subgraph()
result = assignment_editor_graph.invoke(initial_state)

/Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/.venv/lib/python3.12/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


In [10]:
story_brief = result["story_brief"]
print(story_brief.model_dump_json(indent=2))


{
  "topic": "Socotra: Inside the ‘Alien’ Island Where Evolution Went Off-Script",
  "angle": "Use Socotra as a living laboratory of evolution and isolation. Open with a vivid scene—blood-red resin from a dragon’s blood tree on a fog-wrapped plateau or phosphorescent waves on a remote beach—then move outward to explain why this island looks like another planet. Trace how Socotra’s geological history and extreme isolation produced its bizarre flora and fauna, and balance scientific explanation with the lived reality of Socotra’s people. Explore how conflict, tourism, and climate change are converging on this fragile ecosystem, posing a central tension: can Socotra remain otherworldly in a rapidly changing world?",
  "length": 900,
  "key_questions": [
    "What makes Socotra’s landscape and living organisms so strikingly different from almost anywhere else on Earth, from dragon’s blood trees to desert roses and cave-dwelling species?",
    "How did Socotra’s geological and evolutionary 

In [11]:
from IPython.display import Markdown, display

def display_story_brief(brief: StoryBrief):
    """Format and display a StoryBrief as human-friendly markdown."""
    
    # Build the markdown string
    md_parts = []
    
    # Header
    md_parts.append("# 📋 Story Brief")
    md_parts.append("")
    md_parts.append("---")
    md_parts.append("")
    
    # Topic
    md_parts.append("## Topic")
    md_parts.append("")
    md_parts.append(brief.topic)
    md_parts.append("")
    
    # Angle
    md_parts.append("## Angle")
    md_parts.append("")
    md_parts.append(brief.angle)
    md_parts.append("")
    
    # Length
    md_parts.append("## Length")
    md_parts.append("")
    md_parts.append(f"**{brief.length} words**")
    md_parts.append("")
    
    # Key Questions
    md_parts.append(f"## Key Questions ({len(brief.key_questions)})")
    md_parts.append("")
    if brief.key_questions:
        for i, question in enumerate(brief.key_questions, 1):
            md_parts.append(f"{i}. {question}")
    else:
        md_parts.append("*No key questions listed*")
    md_parts.append("")
    
    # Join and display
    markdown_text = "\n".join(md_parts)
    display(Markdown(markdown_text))
    
    return markdown_text

In [12]:
display_story_brief(story_brief)

# 📋 Story Brief

---

## Topic

Socotra: Inside the ‘Alien’ Island Where Evolution Went Off-Script

## Angle

Use Socotra as a living laboratory of evolution and isolation. Open with a vivid scene—blood-red resin from a dragon’s blood tree on a fog-wrapped plateau or phosphorescent waves on a remote beach—then move outward to explain why this island looks like another planet. Trace how Socotra’s geological history and extreme isolation produced its bizarre flora and fauna, and balance scientific explanation with the lived reality of Socotra’s people. Explore how conflict, tourism, and climate change are converging on this fragile ecosystem, posing a central tension: can Socotra remain otherworldly in a rapidly changing world?

## Length

**900 words**

## Key Questions (5)

1. What makes Socotra’s landscape and living organisms so strikingly different from almost anywhere else on Earth, from dragon’s blood trees to desert roses and cave-dwelling species?
2. How did Socotra’s geological and evolutionary history—its plate-tectonic journey, isolation, and unusual climate—create such a high level of endemism and ‘alien’ biodiversity?
3. Who lives on Socotra today, and how have local cultures and traditional practices shaped, protected, or strained the island’s ecosystems over centuries?
4. What scientific discoveries and ongoing research are revealing about Socotra’s role as a natural laboratory for evolution and climate resilience?
5. How are modern pressures—political instability in Yemen, conservation challenges, and rising interest in expedition tourism—reshaping the future of Socotra’s unique environment and communities?


'# 📋 Story Brief\n\n---\n\n## Topic\n\nSocotra: Inside the ‘Alien’ Island Where Evolution Went Off-Script\n\n## Angle\n\nUse Socotra as a living laboratory of evolution and isolation. Open with a vivid scene—blood-red resin from a dragon’s blood tree on a fog-wrapped plateau or phosphorescent waves on a remote beach—then move outward to explain why this island looks like another planet. Trace how Socotra’s geological history and extreme isolation produced its bizarre flora and fauna, and balance scientific explanation with the lived reality of Socotra’s people. Explore how conflict, tourism, and climate change are converging on this fragile ecosystem, posing a central tension: can Socotra remain otherworldly in a rapidly changing world?\n\n## Length\n\n**900 words**\n\n## Key Questions (5)\n\n1. What makes Socotra’s landscape and living organisms so strikingly different from almost anywhere else on Earth, from dragon’s blood trees to desert roses and cave-dwelling species?\n2. How did 

In [13]:
import re

def create_slug(topic: str):
    # Take first 5 words of topic and convert to slug
    words = topic.split()[:5]
    article_slug = "_".join(words).lower()
    # Clean up slug (remove non-alphanumeric except underscores)
    article_slug = re.sub(r'[^a-z0-9_]', '', article_slug)
    return article_slug

def save_story_brief(brief: StoryBrief, article_slug: str = None):
    """
    Save a StoryBrief to a directory in project_root/tmp.
    
    Saves both:
    - story_brief.json (can be loaded back into StoryBrief object)
    - story_brief.md (human-readable markdown version)
    
    Args:
        brief: The StoryBrief to save
        article_slug: Optional slug for the article. If not provided, generates from topic.
    
    Returns:
        Path to the created directory
    """
    # Create tmp directory if it doesn't exist
    tmp_dir = project_root / "tmp"
    tmp_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate article slug if not provided
    if article_slug is None:
        article_slug = create_slug(brief.topic)
    
    # Create article directory
    article_dir = tmp_dir / article_slug
    article_dir.mkdir(exist_ok=True)
    
    # Save JSON (can be loaded back into StoryBrief)
    json_path = article_dir / "story_brief.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(brief.model_dump(), f, indent=2, ensure_ascii=False)
    
    # Save markdown version using display_story_brief
    md_path = article_dir / "story_brief.md"
    # Get markdown content by calling display_story_brief (without emoji header)
    md_content = display_story_brief(brief).replace("# 📋 Story Brief", "# Story Brief")
    with open(md_path, 'w', encoding='utf-8') as f:
        f.write(md_content)
    
    print(f"✅ Story brief saved to: {article_dir}")
    print(f"   - JSON: {json_path.name}")
    print(f"   - Markdown: {md_path.name}")
    
    return article_dir

# Save the story brief (using story_brief from Cell 14)
article_slug = create_slug(story_brief.topic)
saved_dir = save_story_brief(story_brief, article_slug=article_slug)

# 📋 Story Brief

---

## Topic

Socotra: Inside the ‘Alien’ Island Where Evolution Went Off-Script

## Angle

Use Socotra as a living laboratory of evolution and isolation. Open with a vivid scene—blood-red resin from a dragon’s blood tree on a fog-wrapped plateau or phosphorescent waves on a remote beach—then move outward to explain why this island looks like another planet. Trace how Socotra’s geological history and extreme isolation produced its bizarre flora and fauna, and balance scientific explanation with the lived reality of Socotra’s people. Explore how conflict, tourism, and climate change are converging on this fragile ecosystem, posing a central tension: can Socotra remain otherworldly in a rapidly changing world?

## Length

**900 words**

## Key Questions (5)

1. What makes Socotra’s landscape and living organisms so strikingly different from almost anywhere else on Earth, from dragon’s blood trees to desert roses and cave-dwelling species?
2. How did Socotra’s geological and evolutionary history—its plate-tectonic journey, isolation, and unusual climate—create such a high level of endemism and ‘alien’ biodiversity?
3. Who lives on Socotra today, and how have local cultures and traditional practices shaped, protected, or strained the island’s ecosystems over centuries?
4. What scientific discoveries and ongoing research are revealing about Socotra’s role as a natural laboratory for evolution and climate resilience?
5. How are modern pressures—political instability in Yemen, conservation challenges, and rising interest in expedition tourism—reshaping the future of Socotra’s unique environment and communities?


✅ Story brief saved to: /Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/tmp/socotra_inside_the_alien_island
   - JSON: story_brief.json
   - Markdown: story_brief.md


In [15]:
print(article_slug)

socotra_inside_the_alien_island
